In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import setup
import parse
import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.activations import relu
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TerminateOnNaN, EarlyStopping
from sklearn.model_selection import train_test_split, KFold

In [ ]:
df = pd.read_csv(setup.DATASET_DIR+setup.DATASET, nrows=setup.N_ROWS, dtype={"Fen": np.string_, "Evaluation": np.string_})
df

### Vectorize the dataset

Each board is converted from FEN to the following vectorized representation:

TODO

We discard the move counter and half-move clock provided in the last field in the FEN format. This means that we will have to take care of draws manually.

Un-normalized scores can be as high/low as 15k. Each score is converted to a range of -1 - 1 using a sigmoid to reflect the likelyhood of winning.

| **Output type**                 | **Value range** |
|----------------|--------------------------|
| Score             | 0 - 1                    |

In [ ]:
features = [f"f_{str(x)}" for x in range(1, setup.N_FEATURES+1)]
df_vectorized = pd.DataFrame(df["FEN"].apply(lambda fen_str: parse.fen_to_vector(fen_str)).to_list(), columns=features)
df_vectorized["label"] = df.apply(lambda row: parse.convert_stockfish_eval(row.FEN, row.Evaluation), axis=1)
df_vectorized

In [ ]:
# Check for null values
assert df_vectorized[df_vectorized.isnull().values].empty
# Check shape (+1 is for the label)
assert df_vectorized.shape == (setup.N_ROWS, setup.N_FEATURES + 1)

In [ ]:
# Split in training set and cross-validation set (shuffles randomly and splits)
X_train, X_cv, y_train, y_cv = train_test_split(df_vectorized[features], df_vectorized["label"], train_size=setup.TRAINING_SET_SIZE)

### Train the model

In [ ]:
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(X)
'''
TODOs
* Try normalizing and check if we are doing it properly
* Try elu and leakyrelu activations
* Try SGD
'''
def create_model():
    model = Sequential(
        [
            # normalizer,
            Input(shape=(setup.N_FEATURES,)),
            Dense(1024, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
            Dense(1024, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
            Dense(1024, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
            Dense(1, activation=setup.OUTPUT_ACTIVATION),
        ]
    )
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=setup.LEARNING_RATE),
        # optimizer=SGD(learning_rate=setup.LEARNING_RATE, nesterov=True, momentum=0.7),
    )

    return model

model = create_model()
model.summary()

In [ ]:
def train_evaluate_model(model, X_train, y_train, X_cv, y_cv):
    active_callbacks = [TerminateOnNaN()]
    if setup.EARLY_STOPPING:
        # Early stopping on training set loss
        active_callbacks.append(EarlyStopping(monitor="loss", patience=setup.PATIENCE))
        # Early stopping on cross-valiation set loss
        active_callbacks.append(EarlyStopping(monitor="val_loss", patience=setup.PATIENCE))

    history = model.fit(
        X_train, y_train, validation_data=(X_cv, y_cv),
        epochs=setup.EPOCHS, batch_size=setup.BATCH_SIZE, 
        callbacks=active_callbacks
    )

    train_error = model.evaluate(X_train, y_train)
    cv_error = model.evaluate(X_cv, y_cv)
    return history, train_error, cv_error

history, train_error, cv_error = train_evaluate_model(model, X_train, y_train, X_cv, y_cv)

### Evaluate error and plot learning curve

In [ ]:
utils.plot_learning_curve(history)
print(f"Training set error: {train_error:.2}")
print(f"Cross-validation set error: {cv_error:.2}")
print(f"Random classifier error: {((np.random.rand(len(y_cv)) - y_cv)**2).mean():.2}")

In [ ]:
if setup.SAVE_MODEL:
    model.save(setup.MODEL_NAME)
# model = tensorflow.keras.models.load_model(setup.MODEL_NAME)

In [ ]:
if setup.K_FOLD:
    k_fold = KFold(n_splits=setup.N_FOLDS)
    train_error_all = []
    cv_error_all = []
    X = df_vectorized[features]
    y = df_vectorized["label"]

    # TODO reset index first?
    for i, (train, test) in enumerate(k_fold.split(X, y)):
        model = create_model()
        _, train_error, cv_error = train_evaluate_model(model, X.loc[train], y.loc[train], X.loc[test], y.loc[test])
        train_error_all.append(train_error)
        cv_error_all.append(cv_error)
        print(f"Model #{i+1} done! CV error: {cv_error:.2}")
        del model

    print(f"Training sets error:{train_error_all}")
    print(f"Cross-validation sets sets errors:{cv_error_all}")
    print(f"Training sets mean error: {np.mean(np.array(train_error_all)):.2}")
    print(f"Cross-validation sets mean error: {np.mean(np.array(cv_error_all)):.2}")